In [1]:
# Лабораторная работа 1 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель линейной рециркуляционной сети 
# с постоянным коэффициентом обучения и нормированными весовыми коэффициентами.
#
# Данный файл содержит алгоритмы построения графиков различных зависимостей

In [16]:
from lrnn import (
    LRNN,
    image_to_blocks,
    compress_image,
    decompress_image
)
import numpy as np
from PIL import Image
import plotly.graph_objects as go
from random import randint

In [3]:
MAX_RGB_VALUE = 255
COLOR_CHANNELS_AMOUNT = 3

In [4]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 20 

img = Image.open('car.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)
# Compression coeff
print('Z =', (n*l) / ((n+l) * p+2))

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.1))])


network = LRNN(n, p, 0.0015)
network.train(train, 15000, learn_by_loss=True, max_loss=2000)

compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)

compression_info_size = (
    compressed.size * compressed.itemsize +
    network.W_dec.size * network.W_dec.itemsize +
    np.array(shape).size * np.array(shape).itemsize +
    np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
) * 8
print(f'Compression coefficient: {(img_array.size * img_array.itemsize * 8) / compression_info_size}')
print('Z =', (n*l) / ((n+l) * p + 2))

dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg_array = np.asarray(dimg)
dimg.save('compression-decompression_test.jpg')

Z = 4.807507403561401
Epoch 1/15000, Loss: 3059.400024158364
MSE: 4.079200
Epoch 2/15000, Loss: 2557.9787317354717
MSE: 3.410638
Epoch 3/15000, Loss: 2292.3977805440095
MSE: 3.056530
Epoch 4/15000, Loss: 2098.1943998903635
MSE: 2.797593
Epoch 5/15000, Loss: 1946.2904365087186
MSE: 2.595054
TRAINING FINISHED ON:
Epoch 5/15000, Loss: 1946.2904365087186
MSE: 2.595054
Compression coefficient: 1.2334714821393329
Z = 4.807507403561401


## Relation between learning rate and epochs 

In [ ]:
alphas = np.linspace(0.0001, 0.004, endpoint=True, num=10)
all_alphas: list[float] = []
epochs_amount_mean: list[int] = []
epochs_amount: list[int] = []
fig = go.Figure()
experiments_amount = 5
for alpha in alphas:
    print(f'ALPHA: {alpha}')
    epoch_sum = 0
    for i in range(experiments_amount):
        all_alphas.append(alpha)        
        dl = LRNN(n, 20, alpha)
        dl.train(train, 15000, learn_by_loss=True, max_loss=1400, verbosity=20)        
        epoch_sum += dl.epoch
        epochs_amount.append(dl.epoch)
    epochs_amount_mean.append(epoch_sum / experiments_amount)
fig.add_trace(go.Scatter(x=all_alphas, y=epochs_amount, mode='markers',
                         name='Все наблюдения'))
fig.add_trace(go.Scatter(x=alphas, y=epochs_amount_mean, mode='lines+markers',
                         name='Средние значения'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента обучения',
                  xaxis_title='Значение коэффициента обучения',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

ALPHA: 0.0001
Epoch 20/15000, Loss: 2472.598842698493
MSE: 3.296798
Epoch 40/15000, Loss: 2133.64940099485
MSE: 2.844866
Epoch 60/15000, Loss: 1894.5084760299344
MSE: 2.526011
Epoch 80/15000, Loss: 1726.708935222988
MSE: 2.302279
Epoch 100/15000, Loss: 1608.3383361951994
MSE: 2.144451
Epoch 120/15000, Loss: 1518.008913365979
MSE: 2.024012
Epoch 140/15000, Loss: 1444.2790656913367
MSE: 1.925705
TRAINING FINISHED ON:
Epoch 154/15000, Loss: 1399.5325900541445
MSE: 1.866043
Epoch 20/15000, Loss: 2547.80844252467
MSE: 3.397078
Epoch 40/15000, Loss: 2197.8129063704678
MSE: 2.930417
Epoch 60/15000, Loss: 1939.705025801769
MSE: 2.586273
Epoch 80/15000, Loss: 1746.0980617654795
MSE: 2.328131
Epoch 100/15000, Loss: 1612.7475667482597
MSE: 2.150330
Epoch 120/15000, Loss: 1515.9297662383185
MSE: 2.021240
Epoch 140/15000, Loss: 1439.4313506765857
MSE: 1.919242
TRAINING FINISHED ON:
Epoch 152/15000, Loss: 1399.9370241159857
MSE: 1.866583
Epoch 20/15000, Loss: 2585.182472561129
MSE: 3.446910
Epoch 40

## Relationship between hidden layer dimension and epochs

In [6]:
p_s = (6, 8, 10, 12, 14, 16, 18, 20)
epochs_amount: list[int] = []
compression_rates: list[float] = []
for p in p_s:    
    dl = LRNN(n, p, 0.001)
    dl.train(train, 15000, learn_by_loss=True, max_loss=1900)
    epochs_amount.append(dl.epoch)

    compressed = compress_image(dl.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
    compression_info_size = (
        compressed.size * compressed.itemsize +
        network.W_dec.size * network.W_dec.itemsize +
        np.array(shape).size * np.array(shape).itemsize +
        np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
    ) * 8
    compression_rates.append((img_array.size * img_array.itemsize * 8) / compression_info_size)

fig = go.Figure()
fig.add_trace(go.Scatter(x=compression_rates, y=epochs_amount, mode='lines+markers'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента сжатия',
                  xaxis_title='Коэффициент сжатия',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

Epoch 1/15000, Loss: 3871.0088178801075
MSE: 5.161345
Epoch 2/15000, Loss: 3069.319768276927
MSE: 4.092426
Epoch 3/15000, Loss: 2823.2647099728106
MSE: 3.764353
Epoch 4/15000, Loss: 2679.9128621632676
MSE: 3.573217
Epoch 5/15000, Loss: 2583.7914645805495
MSE: 3.445055
Epoch 6/15000, Loss: 2514.77253007027
MSE: 3.353030
Epoch 7/15000, Loss: 2462.8924830227984
MSE: 3.283857
Epoch 8/15000, Loss: 2422.542070471053
MSE: 3.230056
Epoch 9/15000, Loss: 2390.306928550228
MSE: 3.187076
Epoch 10/15000, Loss: 2363.985454155671
MSE: 3.151981
Epoch 11/15000, Loss: 2342.0962581748645
MSE: 3.122795
Epoch 12/15000, Loss: 2323.606638823043
MSE: 3.098142
Epoch 13/15000, Loss: 2307.7748160489386
MSE: 3.077033
Epoch 14/15000, Loss: 2294.055179344929
MSE: 3.058740
Epoch 15/15000, Loss: 2282.0380225755034
MSE: 3.042717
Epoch 16/15000, Loss: 2271.409715542638
MSE: 3.028546
Epoch 17/15000, Loss: 2261.926633726728
MSE: 3.015902
Epoch 18/15000, Loss: 2253.3967669698136
MSE: 3.004529
Epoch 19/15000, Loss: 2245.66

KeyboardInterrupt: 

In [ ]:
p_s = (6, 8, 10, 12, 14, 16, 18, 20)
all_ps: list[int] = []
all_compression_rates: list[float] = []
compression_rates: list[float] = []
epochs_amount_mean: list[int] = []
epochs_amount: list[int] = []
fig = go.Figure()
experiments_amount = 5
for p in p_s:
    epoch_sum = 0
    for i in range(experiments_amount):
        all_ps.append(p)        
        dl = LRNN(n, p, 0.001)
        dl.train(train, 15000, learn_by_loss=True, max_loss=1500)
        epoch_sum += dl.epoch
        epochs_amount.append(dl.epoch)

        compressed = compress_image(dl.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
        compression_info_size = (
            compressed.size * compressed.itemsize +
            network.W_dec.size * network.W_dec.itemsize +
            np.array(shape).size * np.array(shape).itemsize +
            np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
        ) * 8
        all_compression_rates.append((img_array.size * img_array.itemsize * 8) / compression_info_size)
    compression_rates.append(all_compression_rates[-1])

    epochs_amount_mean.append(epoch_sum / experiments_amount)
fig.add_trace(go.Scatter(x=all_compression_rates, y=epochs_amount, mode='markers',
                         name='Все наблюдения'))
fig.add_trace(go.Scatter(x=compression_rates, y=epochs_amount_mean, mode='lines+markers',
                         name='Средние значения'))
fig.update_layout(title='Зависимость количества итераций обучения от коэффициента сжатия',
                  xaxis_title='Значение коэффичиента сжатия',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

## Relationship between max error and epochs

In [ ]:
max_errors = list(range(800, 3100, 100))
all_errors: list[float] = []
epochs_amount_mean: list[int] = []
epochs_amount: list[int] = []
fig = go.Figure()
experiments_amount = 5
for err in max_errors:
    epoch_sum = 0
    for i in range(experiments_amount):
        all_errors.append(err)        
        dl = LRNN(n, 50, 0.001)
        dl.train(train, 15000, learn_by_loss=True, max_loss=err)
        epoch_sum += dl.epoch
        epochs_amount.append(dl.epoch)
    epochs_amount_mean.append(epoch_sum / experiments_amount)
fig.add_trace(go.Scatter(x=all_errors, y=epochs_amount, mode='markers',
                         name='Все наблюдения'))
fig.add_trace(go.Scatter(x=max_errors, y=epochs_amount_mean, mode='lines+markers',
                         name='Средние значения'))
fig.update_layout(title='Зависимость количества итераций обучения от значения максимальной ошибки',
                  xaxis_title='Значение максимальной ошибки',
                  yaxis_title='Количество итераций обучения',
                  height=720, width=1280,
                  font=dict(size=20))
fig.show()

## Relationship between image and epochs

In [ ]:
# Collecting everything

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 20 

img = Image.open('images/winter.jpg')
img_array = np.asarray(img)
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.05))])

network = LRNN(n, p, 0.001)
network.train(train, 15000, learn_by_loss=True, max_loss=1300)

r, g, b = img.split()
fig = go.Figure()
fig.add_trace(
    go.Histogram(x=np.array(r).flatten(),
                 name='Красный канал', marker_color='red',
                 xbins=dict(
                     start=0,
                     end=255,
                     size=7
                 ),
                 opacity=1,
                 )
)
fig.add_trace(
    go.Histogram(x=np.array(g).flatten(),
                 name='Зелёный канал', marker_color='green',
                 xbins=dict(
                     start=0,
                     end=255,
                     size=7
                 ),
                 opacity=0.6,
                 )
)
fig.add_trace(
    go.Histogram(x=np.array(b).flatten(), 
                 name='Синий канал', marker_color='blue',
                 xbins=dict(
                     start=0,
                     end=255,
                     size=7
                 ),
                 opacity=0.3,
                 )
)
fig.update_layout(barmode='stack',
                  title='Гистограмма изображения',                  
                  height=500, width=500,
                  font=dict(size=20))
fig.show()

print(network.epoch)
compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg_array = np.asarray(dimg)
dimg.save('compression-decompression_test_winter.jpg')

# compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
# dimg_array = np.asarray(dimg)
# dimg.save('compression-decompression_test.jpg')

In [ ]:
# Decompressed image quality based on error

block_width = 10
block_height = 10

n = block_height * block_width
# Hidden layer neuron amount
p = 20 

img = Image.open('images/landscape_bmp.bmp')
img_array = np.asarray(img)
img_array = img_array[:, :, :COLOR_CHANNELS_AMOUNT]
shape = img_array.shape
blocks = image_to_blocks(img_array, block_height, block_width, overlap=0)

l = len(blocks)

color_df = ((2 * blocks / MAX_RGB_VALUE) - 1).reshape(len(blocks), -1, 3).transpose(0, 2, 1).reshape(-1, block_height * block_width)
train = np.matrix(color_df[np.random.choice(color_df.shape[0], int(color_df.shape[0] * 0.05))])

network = LRNN(n, p, 0.0002)
network.train(train, 15000, learn_by_loss=True, max_loss=500)

compressed = compress_image(network.W_enc, img_array, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)

compression_info_size = (
    compressed.size * compressed.itemsize +
    network.W_dec.size * network.W_dec.itemsize +
    np.array(shape).size * np.array(shape).itemsize +
    np.array((block_height, block_width)).size * np.array((block_height, block_width)).itemsize 
) * 8
print(f'Compression rate: {(img_array.size * img_array.itemsize * 8) / compression_info_size:.5f}')


dimg = decompress_image(network.W_dec, compressed, shape, COLOR_CHANNELS_AMOUNT, block_height, block_width, 0)
dimg_array = np.asarray(dimg)
dimg.save('compression-decompression_test_landscape.png')

In [ ]:
np.dot([-3, -2, -2, -3, -1, 2], [0, 0, 0, 0, -1, -2])

In [ ]:
0.46 * (1 - 0.8) * -0.9